In [1]:
from collections import namedtuple
import random
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imutils

In [6]:
digitStruct = pd.read_json('digitStruct.json')

digitStruct.set_index('filename', inplace=True)
len(digitStruct.loc['42.png','boxes'])

3

In [3]:
def get_gt_contours(i):
    gt_contours = []
    boxes = digitStruct.loc[f'{str(i)}.png', 'boxes']
    for box in boxes:
        x = box['left']
        y = box['top']
        w = box['width']
        h = box['height']
        gt_contours.append([x, y, x+w, y+h])
    return gt_contours

In [4]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	
    # compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [9]:
recall = 0
for i in range(1,33402):
    img = cv2.imread(f'train/train/{i}.png')
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    bfilter= cv2.bilateralFilter(gray,11,17,17)
    edges = cv2.Canny(bfilter,30,150)
    keypoints= cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours=imutils.grab_contours(keypoints)
    edged_cont = cv2.drawContours(img.copy(), contours, -1, (0,255,0), 1)
    sorted_contours=sorted(contours,key=cv2.contourArea,reverse=True)
    pred_contours = []
    pred_boxes = img.copy()
    for contour in sorted_contours:
        [x, y, w, h] = cv2.boundingRect(contour)
        pred_contours.append([x, y, x+w, y+h])
        cv2.rectangle(pred_boxes, (x,y), (x+w,y+h), (0,255,0), 1)
    pred_contours = [list(x) for x in set([tuple(L) for L in pred_contours])]
    gt_contours = get_gt_contours(i)
    match_contours = []
    for contour in pred_contours:
        for gt_contour in gt_contours:
            iou = bb_intersection_over_union(contour, gt_contour)
            if iou > 0.5:
                match_contours.append({'points': contour, 'iou':iou})
    recall+=len(match_contours) / len(digitStruct.loc[f'{i}.png','boxes'])
recall/=33402
recall

0.6582664710695939